In [23]:
import os
import pandas as pd
import geopandas as gpd
import cdsapi
from datetime import datetime, timedelta

# === SETTINGS ===
outage_file  = "data_collection_by_hollis/correlated_outage/extreme_outage_events.csv"
weather_dir  = "data_collection_by_hollis/correlated_outage/weather_output"
os.makedirs(weather_dir , exist_ok=True)

# Load outage events
df = pd.read_csv(outage_file)

In [24]:
df = df.drop(columns=[
    col for col in df.columns if any(x in col for x in ["STATEFP", "COUNTYFP", "STATE_NAME", "NAME", "lat", "lon"])
])

# Load U.S. county shapefile from Census
shapefile_path = "data_collection_by_hollis/shapefiles/cb_2020_us_county_5m.zip"
counties = gpd.read_file(shapefile_path)
counties["lat"] = counties.geometry.centroid.y
counties["lon"] = counties.geometry.centroid.x

# Re-project to a U.S.-appropriate planar CRS
counties = counties.to_crs(epsg=5070)
counties["county_lat"] = counties.geometry.centroid.to_crs(epsg=4326).y
counties["county_lon"] = counties.geometry.centroid.to_crs(epsg=4326).x

# Rename to match clean merge keys
counties = counties.rename(columns={
    "NAME": "county",
    "STATE_NAME": "state"
})

# Merge on lowercase standardized fields
df = df.merge(
    counties[["state", "county", "county_lat", "county_lon"]],
    on=["state", "county"],
    how="left"
)

counties[['state', 'county', 'county_lat', 'county_lon']].to_csv("county_centroids.csv", index=False)


C:\Users\holli\AppData\Local\Temp\ipykernel_26600\836355668.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lat"] = counties.geometry.centroid.y
C:\Users\holli\AppData\Local\Temp\ipykernel_26600\836355668.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties["lon"] = counties.geometry.centroid.x


In [ ]:
# Function to get full 48-hour data (2 days)
def download_hourly_weather(lat, lon, date_str, save_path):
    client = cdsapi.Client()
    target_date = datetime.strptime(date_str, "%Y-%m-%d")
    
    for offset in [2, 1]:  # 2 days before, then 1 day before
        d = target_date - timedelta(days=offset)
        area = [float(lat)+0.1, float(lon)-0.1, float(lat)-0.1, float(lon)+0.1]

        year, month, day = d.strftime("%Y"), d.strftime("%m"), d.strftime("%d")
        out_file = save_path.replace(".nc", f"_D-{offset}.nc")

        if os.path.exists(out_file):
            print(f"Already downloaded: {out_file}")
            continue

        print(f"Requesting: {out_file}")
        client.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    # Wind
                    '10m_u_component_of_wind', '10m_v_component_of_wind',
                    'maximum_wind_gust_since_previous_post_processing',

                    # Temperature
                    '2m_temperature',
                    'maximum_2m_temperature_since_previous_post_processing',
                    'minimum_2m_temperature_since_previous_post_processing',
                    'dew_point_temperature',
                    'skin_temperature',

                    # Ice & Cold
                    'total_precipitation',
                    'snowfall',

                    # Storms & Pressure
                    'mean_sea_level_pressure',
                    'surface_pressure',

                    # Fire/Drought risk
                    'surface_solar_radiation_downwards',
                    'potential_evaporation',
                    'evaporation',
                    'total_column_water_vapour',
                    'convective_available_potential_energy',
                ],
                'year': year,
                'month': month,
                'day': day,
                'time': [f"{h:02}:00" for h in range(24)],  # hourly times
                'area': area,
            },
            out_file
        )

In [ ]:
# Download for each outage
for i, row in df.iterrows():
    try:
        date = pd.to_datetime(row['datetime_event_began']).strftime('%Y-%m-%d')
        lat, lon = row['county_lat'], row['county_lon']
        base_path = os.path.join(weather_dir, f"event_{i}_{date}.nc")
        download_hourly_weather(lat, lon, date, base_path)
    except Exception as e:
        print(f"Failed event {i} ({row.get('county', '')}): {e}")

2025-04-23 12:38:24,846 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-23 12:38:24,847 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Requesting: data_collection_by_hollis/correlated_outage/weather_output\event_0_2014-11-24_D-2.nc
Failed event 0 (Clay): 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
required licences not accepted
Not all the required licences have been accepted; please visit https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download#manage-licences to accept the required licence(s).


2025-04-23 12:38:31,021 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-23 12:38:31,022 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Requesting: data_collection_by_hollis/correlated_outage/weather_output\event_1_2014-11-24_D-2.nc
Failed event 1 (Crittenden): 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
required licences not accepted
Not all the required licences have been accepted; please visit https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download#manage-licences to accept the required licence(s).


2025-04-23 12:38:32,132 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-23 12:38:32,133 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


...

2025-04-23 12:39:14,582 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-23 12:39:14,584 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Requesting: data_collection_by_hollis/correlated_outage/weather_output\event_30_2014-11-26_D-2.nc
Failed event 30 (Hartford): 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
required licences not accepted
Not all the required licences have been accepted; please visit https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download#manage-licences to accept the required licence(s).


2025-04-23 12:39:15,668 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-23 12:39:15,669 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Requesting: data_collection_by_hollis/correlated_outage/weather_output\event_31_2014-11-26_D-2.nc


Recovering from HTTP error [429 Too Many Requests], attempt 1 of 500
Retrying in 120 seconds
